In [1]:
import sys
sys.path.append('../src')


In [2]:
from tensorflow import keras
import tensorflow as tf
import pandas as pd
import numpy as np
import os


In [3]:
from gait.utils import compute_resultant_acceleration, compute_resultant_gyro, compute_resultant_angle, get_overlap_data_all_sessions, create_dir
from gait.training import change_x_lstm_shape


In [4]:
def change_shape_lstm(X):
    n_steps, n_length = 4, 32
    return X.reshape((X.shape[0], n_steps, n_length, 12))


In [5]:
OVERLAP_PERCENT = 80
# overlapPercents = [0]
exclude_subjects = ['ddAeJA42PXvwthbW', 'nan', 'sUZBISq61Y7I5tqQ', 'LLZjAPTyj7muHsEf',
                    'cbOZWnI7s1y8oLD4', 'EUbKPOSQgjccjtvi', 'MMuX9YIh4NTbLZLM',
                    'PE8D53oX060qLbdX', 'ddAeJA42PXvwthbW', 'xYdtS1F8tDyjEIgN', '19AoxD1bgrDckd2p',
                    'wtyNo4LYaWXrkzA7']

X, y, subjects = get_overlap_data_all_sessions(OVERLAP_PERCENT)
# REMOVE UNWANTED SUBJECTS
indexes = np.where(subjects == exclude_subjects)
X = np.delete(X, indexes[0], axis=0)
y = np.delete(y, indexes[0], axis=0)
subjects = np.delete(subjects, indexes[0], axis=0)
# END REMOVE UNWANTED SUBJECTS
# AUGMENT DATA
resultant_acc = compute_resultant_acceleration(X)
resultant_gyro = compute_resultant_gyro(X)
resultant_angle = compute_resultant_angle(X)
resultant_acc = resultant_acc.reshape(
    resultant_acc.shape[0], resultant_acc.shape[1], 1)
resultant_gyro = resultant_gyro.reshape(
    resultant_gyro.shape[0], resultant_gyro.shape[1], 1)
resultant_angle = resultant_angle.reshape(
    resultant_angle.shape[0], resultant_angle.shape[1], 1)
X = np.concatenate((X, resultant_acc), axis=2)
X = np.concatenate((X, resultant_gyro), axis=2)
X = np.concatenate((X, resultant_angle), axis=2)
# AUGMENT DATA END


In [17]:
trainX = change_x_lstm_shape(X)
representative_dataset = trainX.astype(np.float32)



In [18]:
print(X.shape)

(80532, 128, 12)


In [19]:
tf.data.Dataset.from_tensor_slices((representative_dataset)).batch(10).take(1000)


<TakeDataset element_spec=TensorSpec(shape=(None, 128, 12), dtype=tf.float32, name=None)>

In [20]:
model_path = '../models/model_80_overlap/best_model.75-0.05-0.9856074452400208.hdf5'
loaded_model = keras.models.load_model(model_path)

QUANTIZED_ROOT = './quantized'
tflite_model_name = 'model.tflite'
quantized_model_name = 'model_quantized.tflite'
print(model_path.split('/')[3])
model_paths = model_path.split('/')

tflite_path = os.path.join(QUANTIZED_ROOT, *model_paths[2:-1])
tflite_filepath = os.path.join(
    QUANTIZED_ROOT, *model_paths[2:-1], tflite_model_name)
create_dir(tflite_path)

converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite_model = converter.convert()
open(tflite_filepath, 'wb').write(tflite_model)



best_model.75-0.05-0.9856074452400208.hdf5


INFO:tensorflow:Assets written to: /tmp/tmppcvirzcw/assets


INFO:tensorflow:Assets written to: /tmp/tmppcvirzcw/assets


565516

In [21]:
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices((representative_dataset)).batch(1).take(100):
    # Model has only one input so each data point has one element.
        print(input_value.shape)
        yield input_value

In [26]:
trainX = change_x_lstm_shape(X)
representative_dataset = np.array(trainX.astype(np.float32))

In [27]:
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(representative_dataset).batch(1).take(1):
        yield [input_value]

model_path = '../models/model_80_overlap/best_model.75-0.05-0.9856074452400208.hdf5'
loaded_model = keras.models.load_model(model_path)

converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS] 
converter._experimental_lower_tensor_list_ops = False
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant = converter.convert()


INFO:tensorflow:Assets written to: /tmp/tmp817jjekr/assets


INFO:tensorflow:Assets written to: /tmp/tmp817jjekr/assets
/home/sanjeev/.local/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


ConverterError: /home/sanjeev/.local/lib/python3.9/site-packages/tensorflow/python/saved_model/save.py:1369:0: error: 'tf.TensorListReserve' op is neither a custom op nor a flex op
<unknown>:0: note: loc(fused["StatefulPartitionedCall:", "StatefulPartitionedCall"]): called from
/home/sanjeev/.local/lib/python3.9/site-packages/tensorflow/python/saved_model/save.py:1369:0: note: Error code: ERROR_NEEDS_FLEX_OPS
/home/sanjeev/.local/lib/python3.9/site-packages/tensorflow/python/saved_model/save.py:1369:0: error: 'tf.TensorListFromTensor' op is neither a custom op nor a flex op
<unknown>:0: note: loc(fused["StatefulPartitionedCall:", "StatefulPartitionedCall"]): called from
/home/sanjeev/.local/lib/python3.9/site-packages/tensorflow/python/saved_model/save.py:1369:0: note: Error code: ERROR_NEEDS_FLEX_OPS
/home/sanjeev/.local/lib/python3.9/site-packages/tensorflow/python/saved_model/save.py:1369:0: error: 'tf.TensorListStack' op is neither a custom op nor a flex op
<unknown>:0: note: loc(f

In [22]:
model_path = '../models/model_80_overlap/best_model.75-0.05-0.9856074452400208.hdf5'
loaded_model = keras.models.load_model(model_path)

converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter._experimental_lower_tensor_list_ops = False
tflite_quant_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp52xu6ew6/assets


INFO:tensorflow:Assets written to: /tmp/tmp52xu6ew6/assets
/home/sanjeev/.local/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


TypeError: 'numpy.ndarray' object is not callable

In [23]:
model_path = '../models/model_80_overlap/best_model.75-0.05-0.9856074452400208.hdf5'
loaded_model = keras.models.load_model(model_path)
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite_model = converter.convert()
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.experimental_new_converter=True
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
tf.lite.OpsSet.SELECT_TF_OPS]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
tflite_quantized_model = converter.convert()

tflite_quantized_model_filtepath =  os.path.join(
    QUANTIZED_ROOT, *model_paths[2:-1], quantized_model_name)

open(tflite_quantized_model_filtepath, "wb").write(tflite_model)
basic_model_size = os.path.getsize(tflite_model)

print("Basic model is %d bytes" % basic_model_size)
quantized_model_size = os.path.getsize(tflite_quantized_model)
print("Quantized model is %d bytes" % quantized_model_size)
difference = basic_model_size - quantized_model_size
print("Difference is %d bytes" % difference)

INFO:tensorflow:Assets written to: /tmp/tmpq0jr6igk/assets


INFO:tensorflow:Assets written to: /tmp/tmpq0jr6igk/assets


INFO:tensorflow:Assets written to: /tmp/tmp0ayyiy13/assets


INFO:tensorflow:Assets written to: /tmp/tmp0ayyiy13/assets
/home/sanjeev/.local/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


In [13]:
keras_file = "../keras_models/model_80_overlap/best_model.75-0.05-0.9856074452400208.hdf5"
loaded_model = keras.models.load_model(model_path)
tf.keras.models.save_model(loaded_model, keras_file)


In [15]:
model_path = '../models/model_80_overlap/best_model.75-0.05-0.9856074452400208.hdf5'
loaded_model = keras.models.load_model(model_path)
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite_model = converter.convert()
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
tflite_quantized_model = converter.convert()

tflite_quantized_model_filtepath =  os.path.join(
    QUANTIZED_ROOT, *model_paths[2:-1], quantized_model_name)

open(tflite_quantized_model_filtepath, "wb").write(tflite_model)
basic_model_size = os.path.getsize(tflite_model)

print("Basic model is %d bytes" % basic_model_size)
quantized_model_size = os.path.getsize(tflite_quantized_model)
print("Quantized model is %d bytes" % quantized_model_size)
difference = basic_model_size - quantized_model_size
print("Difference is %d bytes" % difference)

INFO:tensorflow:Assets written to: /tmp/tmplzz2sla4/assets


INFO:tensorflow:Assets written to: /tmp/tmplzz2sla4/assets


INFO:tensorflow:Assets written to: /tmp/tmpkodffj_n/assets


INFO:tensorflow:Assets written to: /tmp/tmpkodffj_n/assets
/home/sanjeev/.local/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


ConverterError: /home/sanjeev/.local/lib/python3.9/site-packages/tensorflow/python/saved_model/save.py:1369:0: error: 'tf.TensorListReserve' op requires element_shape to be static during TF Lite transformation pass
<unknown>:0: note: loc(fused["StatefulPartitionedCall:", "StatefulPartitionedCall"]): called from
/home/sanjeev/.local/lib/python3.9/site-packages/tensorflow/python/saved_model/save.py:1369:0: error: failed to legalize operation 'tf.TensorListReserve' that was explicitly marked illegal
<unknown>:0: note: loc(fused["StatefulPartitionedCall:", "StatefulPartitionedCall"]): called from
<unknown>:0: error: Lowering tensor list ops is failed. Please consider using Select TF ops and disabling `_experimental_lower_tensor_list_ops` flag in the TFLite converter object. For example, converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]\n converter._experimental_lower_tensor_list_ops = False
